### 0. 필요한 라이브러리 Import

In [1]:
import os
import pandas as pd
import pandas_profiling
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import kss
from konlpy.tag import Kkma  
from konlpy.tag import Twitter

# %appdata%\nltk_data\corpora\stopwords
# korean 파일을 생성하여 복사함
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

np.random.seed(42)

twitter = Twitter()
kkma = Kkma()

# 한글출력
# matplotlib.rc('font', family='AppleGothic')  # MacOS
matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

### 1. 통합된 원본 데이터 파일을 읽어온다

In [2]:

ROOT_DIR = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
dataset = pd.read_csv(os.path.join(ROOT_DIR, "dataset/one.csv"))

print(dataset.shape)
print(dataset.info())

(50568, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50568 entries, 0 to 50567
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   제목                              50568 non-null  object 
 1   저자                              50568 non-null  object 
 2   발행기관                            50568 non-null  object 
 3   학술지명                            50568 non-null  object 
 4   권호사항                            50568 non-null  object 
 5   발행연도                            50568 non-null  int64  
 6   작성언어                            50568 non-null  object 
 7   주제어                             26933 non-null  object 
 8   KDC                             11761 non-null  float64
 9   등재정보                            13083 non-null  object 
 10  자료형태                            50568 non-null  object 
 11  발행기관 URL                        22715 non-null  object 
 12  수록면                 

In [3]:
dataset.head()

,제목,저자,발행기관,학술지명,권호사항,발행연도,작성언어,주제어,KDC,등재정보,...,발행기관 URL,수록면,KCI 피인용횟수,비고,중단사유,제공처,소장기관,국문 초록 (Abstract),다국어 초록 (Multilingual Abstract),목차
0,전기 이륜자동차 분류기준 현황 : 자동차관리법 중심,정윤재,한국자동차공학회,한국자동차공학회 Workshop,Vol.2011No.9[2011]\r\n,2011,Korean,NaN,556.0,NaN,...,http://www.ksae.org,1-14,NaN,NaN,NaN,DBpia,NaN,NaN,NaN,NaN
1,자동차 사회의 환경과 에너지 문제 = 자동차 엔진 기술의 역사적 동향,김영길(Yong Kil Kim),한국자동차공학회,한국자동차공학회 학술대회 및 전시회,Vol.2014No.11[2014]\r\n,2014,Korean,"Internal Combustion Engine(내연기관),Energy (에너지),...",NaN,NaN,...,NaN,1477-1477,NaN,NaN,NaN,DBpia,NaN,NaN,NaN,NaN
2,수소 연료전지 자동차의 충돌 및 화재안전성 관련 연구동향 = Trends on Cr...,"임재문(Jae Moon Lim),장형진(Hyung Jin Chang),김규현(Gyu...",한국자동차공학회,한국자동차공학회 춘 추계 학술대회 논문집,Vol.-No.-[2008]\r\n,2008,Korean,"Hydrogen Fuel Cell Vehicle(수소 연료전지 자동차),Crash ...",NaN,NaN,...,http://www.ksae.org,1253-1258,NaN,NaN,NaN,DBpia,NaN,환경문제에 따른 배기가스 배출저감 및 자원고갈에 따른 대안으로 향후 수소 연료전지 ...,The researches on the crash and the fire safet...,Abstract\n1. 서론\n2. 수소의 특성\n3. 충돌 및 화재안전성 관련 연...
3,차세대 자동차 개발의 Plastic Hood 설계 및 CAE해석 = Plastic ...,"황태원(Taewon Hwang),최치훈(Chihoon Choi),박상선(Sangsu...",한국자동차공학회,한국자동차공학회 춘 추계 학술대회 논문집,Vol.-No.-[2006]\r\n,2006,Korean,"HOOD(본넷트),Distance(이동량),LS-DYNA(구조해석),MOLDFLOW...",NaN,NaN,...,http://www.ksae.org,2032-2032,NaN,NaN,NaN,DBpia,NaN,현재 생활 필수품 중의 하나인 자동차논 많은 변화를 보이고 있다. 발전사를 살펴보면...,NaN,NaN
4,자동차 산업의 역사와 급변하는 기술에 대응한 교육의 바람직한 방향에 관한 연구 = ...,황경수(Kyungsoo Whang),한국자동차공학회,한국자동차공학회 Workshop,Vol.2001No.-[2001]\r\n,2001,Korean,NaN,NaN,NaN,...,http://www.ksae.org,70-80,NaN,NaN,NaN,DBpia,NaN,오늘날 급속히 변화하고 있는 자동차 기술에 대응한 자동차 기술교육의 방법을 모색하여...,NaN,초록<br/>\n1.서론<br/>\n2.국내 자동차기술교육의 역사<br/>\n3.민...


In [ ]:
dataset['작성언어'].value_counts()

In [26]:
#분석대상 항목으로 발행연도, 제목, 주제어, 국문 초록 (Abstract)을 선택하고 컬럼명을 영문으로 변경함

dataset_1 = dataset.loc[:, ['발행연도', '제목', '주제어', '국문 초록 (Abstract)', '다국어 초록 (Multilingual Abstract)']]
dataset_1 = dataset_1.rename({'발행연도':'year', '제목':'title', '주제어':'keyword', '국문 초록 (Abstract)':'abstract_old', '다국어 초록 (Multilingual Abstract)':'others'}, axis=1)

In [5]:
#-*- coding: utf-8 -*-
import json
import urllib.request
 
client_id = "_PyD14JHCVWfVNEcoGUr" # 개발자센터에서 발급받은 Client ID 값
client_secret = "BQPtnH8S5X" # 개발자센터에서 발급받은 Client Secret 값
 

In [22]:
def translate_test(sentence) :
    if sentence :
        print("not null")

In [7]:
def translate(sentence) :

    result = ""
 
    if sentence :
        encText = urllib.parse.quote(sentence)
        data = "source=en&target=ko&text=" + encText
        url = "https://openapi.naver.com/v1/papago/n2mt"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request, data=data.encode("utf-8"))
        rescode = response.getcode()
        
        result = ""
        if(rescode==200):
            response_body = response.read()
            result = json.loads(response_body)['message']['result']['translatedText']
        else:
            print("Error Code:" + rescode)
    
    return result 

In [14]:
dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50568 entries, 0 to 50567
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          50568 non-null  int64 
 1   title         50568 non-null  object
 2   keyword       26933 non-null  object
 3   abstract_old  10447 non-null  object
 4   others        25062 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [27]:
#국문 초록 값이 없는 건수가 전체의 80%에 해당하고 있음(잘못된 논리임)
#하지만 제목은 초록의 함축적인 문장으로 볼 수 있기 때문에 초록이 없는 경우 제목으로 대체


#0.주제어 예측
#1.가짜 키워드~ 판별....
#2.문서요약
#-키워드 백터
# dataset_2['keyword'].apply(get_words)
dataset_1['abstract'] = np.where(dataset_1['abstract_old'].isnull(), dataset_1['others'], dataset_1['abstract_old'])
#dataset_1['abstract'] = np.where(dataset_1['abstract_old'].isnull(), dataset_1['others'].apply(translate_test), dataset_1['abstract_old'])

del dataset_1['abstract_old']
del dataset_1['others']

dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50568 entries, 0 to 50567
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      50568 non-null  int64 
 1   title     50568 non-null  object
 2   keyword   26933 non-null  object
 3   abstract  28871 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [ ]:
dataset_1.info()

In [ ]:
dataset_1[dataset_1['abstract'].isnull() == False]

In [ ]:
dataset_1.head()

In [ ]:
#지금까지 작업된 정보를 ready_50568.csv 파일로 저장함

dataset_1.to_csv(os.path.join(ROOT_DIR, 'dataset/ready_50568.csv'))

### 2. keyword(주제어)가 없는 데이터 삭제 (26,933개) 

In [ ]:
dataset_2 = dataset_1.loc[:, ['year', 'title', 'keyword', 'abstract']].dropna()
dataset_2.info()

In [ ]:
#keyword에서 영문제거하고, 형태소(명사)를 분리하여 배열로 재구성)
#keyword = []
#for idx, val in enumerate(dataset_2['keyword']) :
#    words = [word for word, tag in kkma.pos(val)
#                if len(word) > 1 and tag in ['NNG', 'NNP', 'NP']]
#    keyword.append(words)

def get_words(row) :
    return [word for word, tag in kkma.pos(row)
                if len(word) > 1 and tag in ['NNG', 'NNP', 'NP']]


dataset_2['keyword'] = dataset_2['keyword'].apply(get_words)
dataset_2.head()

In [ ]:
#기존 키워드를 삭제하고 한글 명사로 재구성한 키워드(주제어)의 컬럼명을 변경함
del dataset_2['keyword']
dataset_2 = dataset_2.rename({'keyword_new':'keyword'}, axis=1)

In [ ]:
#주제어가 있는 가공된 데이터를 저장함
dataset_3.to_csv(os.path.join(ROOT_DIR, 'dataset/ready_26933.csv'))

In [ ]:
#2008년 이상의 데이터만 필터링
condition_by_year = dataset_1['year'] >= 2008
dataset_2 = dataset_1[condition_by_year]
dataset_2 = dataset_2.drop('abstract_old', axis=1)
dataset_2.info()

In [ ]:
profile = dataset_2.profile_report()

In [ ]:
profile.to_file("./report.html")

In [ ]:
#제목으로 주제어를 정하고, 빈도수,
#주제어의 단어 빈도수를 측정해 보자
#멀티 라벨을 OneHot인코딩하여~ 만든다.

In [ ]:
#문장으로 분리

abstracts = dataset_2['abstract'].tolist()
rawdata = []

for abstract in abstracts:
    sentence = kss.split_sentences(abstract)
    rawdata.extend(sentence)

In [29]:
stop_words_eng = stopwords.words('english')
stop_words_kor = stopwords.words('korean')

stop_words = []
#stop_words.extend(stop_words_kor)
#stop_words.extend(stop_words_eng)

stop_words[:5]

[]

In [30]:
#색인 추출
def tokenizer_by_twitter(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stop_words):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
    ]
 
#색인 추출
def tokenizer_by_kkma(raw, pos=["NNG","NNP","NP", "VV"], stopword=stop_words):
    return [
        word for word, tag in kkma.pos(raw)
            if len(word) > 1 and tag in pos and word not in stopword
    ]

In [61]:
dataset_5 = dataset_1[dataset_1['year'] >= 2010]
rawdata = dataset_5["keyword"].dropna().tolist()

In [62]:
len(rawdata)

20425

In [78]:
result = []

for idx, val in enumerate(rawdata):
        if idx % 1000 == 0 :
        print(f'{idx + 1} 개를 처리중입니다. [{val}]')
    words = tokenizer_by_kkma(val)
    result.extend(words)

In [87]:
len(result)

160764

In [88]:
count = Counter(result)

In [96]:
words = dict(count.most_common(100))
print(words)


{'자동차': 4123, '시스템': 2152, '차량': 1782, '해석': 1610, '주행': 1602, '제어': 1473, '장치': 1278, '전기': 1113, '시험': 1054, '연료': 1042, '모델': 965, '자율': 927, '엔진': 891, '분석': 865, '요소': 862, '가스': 726, '충돌': 712, '하이브리드': 655, '설계': 654, '센서': 638, '평가': 617, '분사': 602, '안전': 551, '성능': 547, '산업': 543, '보험': 537, '특성': 528, '구조': 514, '모드': 508, '모터': 504, '촉매': 494, '도로': 490, '변속기': 484, '조향': 482, '자동': 479, '디젤': 476, '최적화': 472, '제동': 468, '유체': 444, '환경': 434, '배출': 431, '교통': 431, '전산': 430, '전지': 429, '토크': 425, '계수': 410, '역학': 410, '에너지': 401, '시뮬레이션': 400, '배터리': 397, '압력': 392, '복합': 390, '물질': 389, '질소': 388, '밸브': 382, '산화물': 379, '운전자': 379, '속도': 374, '마찰': 369, '클러치': 360, '책임': 360, '부품': 357, '디자인': 356, '소음': 348, '연소': 348, '운전': 343, '기술': 340, '브레이크': 338, '사고': 330, '시간': 327, '전자': 326, '수소': 323, '모형': 322, '구동': 315, '시트': 310, '동역학': 308, '최적': 299, '모듈': 295, '전략': 293, '공기': 292, '온도': 291, '실험': 291, '냉각': 290, '가솔린': 290, '강도': 281, '기반': 278, '정보': 277, '펌프': 274, '

In [97]:
words = dict(count.most_common(200))
print(words)

{'자동차': 4123, '시스템': 2152, '차량': 1782, '해석': 1610, '주행': 1602, '제어': 1473, '장치': 1278, '전기': 1113, '시험': 1054, '연료': 1042, '모델': 965, '자율': 927, '엔진': 891, '분석': 865, '요소': 862, '가스': 726, '충돌': 712, '하이브리드': 655, '설계': 654, '센서': 638, '평가': 617, '분사': 602, '안전': 551, '성능': 547, '산업': 543, '보험': 537, '특성': 528, '구조': 514, '모드': 508, '모터': 504, '촉매': 494, '도로': 490, '변속기': 484, '조향': 482, '자동': 479, '디젤': 476, '최적화': 472, '제동': 468, '유체': 444, '환경': 434, '배출': 431, '교통': 431, '전산': 430, '전지': 429, '토크': 425, '계수': 410, '역학': 410, '에너지': 401, '시뮬레이션': 400, '배터리': 397, '압력': 392, '복합': 390, '물질': 389, '질소': 388, '밸브': 382, '산화물': 379, '운전자': 379, '속도': 374, '마찰': 369, '클러치': 360, '책임': 360, '부품': 357, '디자인': 356, '소음': 348, '연소': 348, '운전': 343, '기술': 340, '브레이크': 338, '사고': 330, '시간': 327, '전자': 326, '수소': 323, '모형': 322, '구동': 315, '시트': 310, '동역학': 308, '최적': 299, '모듈': 295, '전략': 293, '공기': 292, '온도': 291, '실험': 291, '냉각': 290, '가솔린': 290, '강도': 281, '기반': 278, '정보': 277, '펌프': 274, '

In [106]:
dataset_5 = dataset_5.dropna()
for word in words :
    count = len(dataset_5[dataset_5['abstract'].str.contains(word)])
    print(f'주제어: {word} / 논문수: {count}')

주제어: 자동차 / 논문수: 5604
주제어: 시스템 / 논문수: 1377
주제어: 차량 / 논문수: 1393
주제어: 해석 / 논문수: 713
주제어: 주행 / 논문수: 939
주제어: 제어 / 논문수: 515
주제어: 장치 / 논문수: 629
주제어: 전기 / 논문수: 802
주제어: 시험 / 논문수: 430
주제어: 연료 / 논문수: 403
주제어: 모델 / 논문수: 810
주제어: 자율 / 논문수: 532
주제어: 엔진 / 논문수: 329
주제어: 분석 / 논문수: 2910
주제어: 요소 / 논문수: 800
주제어: 가스 / 논문수: 470
주제어: 충돌 / 논문수: 162
주제어: 하이브리드 / 논문수: 149
주제어: 설계 / 논문수: 966
주제어: 센서 / 논문수: 277
주제어: 평가 / 논문수: 1038
주제어: 분사 / 논문수: 54
주제어: 안전 / 논문수: 908
주제어: 성능 / 논문수: 893
주제어: 산업 / 논문수: 1558
주제어: 보험 / 논문수: 349
주제어: 특성 / 논문수: 1369
주제어: 구조 / 논문수: 988
주제어: 모드 / 논문수: 112
주제어: 모터 / 논문수: 144
주제어: 촉매 / 논문수: 91
주제어: 도로 / 논문수: 1021
주제어: 변속기 / 논문수: 52
주제어: 조향 / 논문수: 61
주제어: 자동 / 논문수: 5653
주제어: 디젤 / 논문수: 107
주제어: 최적화 / 논문수: 197
주제어: 제동 / 논문수: 94
주제어: 유체 / 논문수: 47
주제어: 환경 / 논문수: 1495
주제어: 배출 / 논문수: 414
주제어: 교통 / 논문수: 894
주제어: 전산 / 논문수: 63
주제어: 전지 / 논문수: 234
주제어: 토크 / 논문수: 73
주제어: 계수 / 논문수: 192
주제어: 역학 / 논문수: 77
주제어: 에너지 / 논문수: 516
주제어: 시뮬레이션 / 논문수: 309
주제어: 배터리 / 논문수: 173
주제어: 압력 / 논문수: 190
주제어: 복합 / 논문수: 293

In [63]:
vect = CountVectorizer(min_df=3, tokenizer=tokenizer_by_kkma)
result = vect.fit_transform(rawdata)
print(
    'fit_transform, (sentence {}, feature {})'.format(result.shape[0], result.shape[1])
)

fit_transform, (sentence 20425, feature 4706)


In [77]:
result = vect.vocabulary_

def f1(x):
    return x[1]

sorted_result = sorted(result.items(), key=f1, reverse=True)
sorted_result

673

In [ ]:
# 문장에서 뽑아낸 feature 들의 Dictionary
features = vect.get_feature_names()
print(features)

In [ ]:
print(result.toarray())

In [ ]:
vectorize = TfidfVectorizer(
    tokenizer=tokenizer_by_twitter,
    min_df=5,
    sublinear_tf = True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata[:4001])
print(
    'fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1])
)
print(X.toarray())
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

In [ ]:
print(X.toarray()[0])

In [ ]:
#키워드 를 축소 필요 
#키워드별로 논문 개수 (논문편수 5개 이하는 제외하자)
#https://codechacha.com/ko/python-google-translate/

- from googletrans import Translator
- translator = Translator()
- result = translator.translate('안녕하세요.', dest="ja")
- print(result[0].text)

#### 구글 번역 API가 변경되어 다음과 같은 설치 필요

- $ pip uninstall googletrans
- $ git clone https://github.com/alainrouillon/py-googletrans.git
- $ cd ./py-googletrans
- $ git checkout origin/feature/enhance-use-of-direct-api
- $ python setup.py install

In [ ]:
#1.주제어 COUNT
#2.분석대상 명확하게~ 
#3.주제어별 관계도